<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/POC_FineTuning_LLM_Mistral_7B_Instruct_v0_1_for_text_to_SQL_EVALDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 , L4  IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


!pip install peft --quiet
!pip install trl ninja packaging --quiet
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

!pip install mistral_inference -q

!pip install trl==0.8.6 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf

In [2]:
#!pip install diffusers safetensors  --quiet
#!pip install colab-env --quiet

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [3]:
from huggingface_hub import login

#print(access_token_write)

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
#from huggingface_hub import notebook_login
#notebook_login(write_permission=True)

In [5]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline

)
from trl import SFTTrainer

In [6]:
# set device
device = 'cuda'

In [7]:
torch.__version__

'2.3.0+cu121'

In [8]:
!apt-get update && apt-get install -y cuda-11-0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [929 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,924 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://archive.ubuntu.com/ubuntu

In [9]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Wed Jun 26 01:02:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   51C    P8       

In [10]:
# First import your own dataset in the default folder which "content" on colab
# The dataset should have one column named "text" with one example per line
data_files = {'train': "/content/gdrive/MyDrive/datasets/train.csv", 'test': "/content/gdrive/MyDrive/datasets/test.csv"}
dataset0 = load_dataset('csv', data_files=data_files)
# Verify the chat template and apply it to you data
# tokenizer.apply_chat_template(chat, tokenize=False)
# Otherwise you can use dataset that are present on https://huggingface.co/datasets
# dataset = load_dataset({DATASET_PATH})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
print(data_files)

{'train': '/content/gdrive/MyDrive/datasets/train.csv', 'test': '/content/gdrive/MyDrive/datasets/test.csv'}


In [12]:
### conversational format
#{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
#{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

In [13]:
!pip install  --upgrade transformers datasets -q

In [14]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

# Load dataset from the hub and processing
dataset = load_dataset("b-mc2/sql-create-context", split="train")
#dataset0 = dataset.train_test_split(test_size=0.2)
# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.shuffle().select(range(22500))
dataset0 = dataset.train_test_split(test_size=2500/22500)
dataset = dataset0.map(create_conversation, remove_columns=dataset.features,batched=False)
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1190457

In [15]:
dataset0

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 2500
    })
})

In [16]:
print(dataset["train"][345]["messages"])

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_57 (total VARCHAR, gold VARCHAR, silver VARCHAR, bronze VARCHAR)', 'role': 'system'}, {'content': 'Name the total number of total with silver of 6, bronze more than 5 and gold less than 4', 'role': 'user'}, {'content': 'SELECT COUNT(total) FROM table_name_57 WHERE silver = 6 AND bronze > 5 AND gold < 4', 'role': 'assistant'}]


In [17]:
from datasets import load_dataset

# Load jsonl data from disk for sql
dataset = load_dataset("json", data_files="train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
dataset

Dataset({
    features: ['messages'],
    num_rows: 20000
})

In [19]:
print(dataset[345]["messages"])

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_57 (total VARCHAR, gold VARCHAR, silver VARCHAR, bronze VARCHAR)', 'role': 'system'}, {'content': 'Name the total number of total with silver of 6, bronze more than 5 and gold less than 4', 'role': 'user'}, {'content': 'SELECT COUNT(total) FROM table_name_57 WHERE silver = 6 AND bronze > 5 AND gold < 4', 'role': 'assistant'}]


In [20]:
dataset_test = load_dataset("json", data_files="test_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
dataset_test

Dataset({
    features: ['messages'],
    num_rows: 2500
})

In [22]:
print(dataset_test[345]["messages"])

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_48 (opera VARCHAR, firefox VARCHAR)', 'role': 'system'}, {'content': 'What opera has 19.87% as the firefox?', 'role': 'user'}, {'content': 'SELECT opera FROM table_name_48 WHERE firefox = "19.87%"', 'role': 'assistant'}]


In [23]:
!pip install huggingface_hub -q

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format


# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.3" #24 JUNE 2024

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# set chat template to OAI chatML, remove if you start from a fine-tuned model
# Set chat template to OAI ChatML after formatting the dataset
model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [25]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32770, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
   

In [26]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [29]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/model/POC-Mistral-7B-text-to-sql-flash-attention-2-dataeval",    # directory to save and repository id

    num_train_epochs=3,                    # number of training epochs
    per_device_train_batch_size=4,          # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    #optim="adamw_torch_fused",              # use fused adamw optimizer
    optim="paged_adamw_32bit",
    logging_steps=10,                       # log every 50 steps
    #save_strategy="epoch",                 # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    weight_decay=0.01,
    #lr_scheduler_type="constant",           # use constant learning rate scheduler
    lr_scheduler_type="cosine",
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    #hub_token=access_token_write,
    load_best_model_at_end=True,
    logging_dir="/content/gdrive/MyDrive/model/POC-Mistral-7B-text-to-sql-flash-attention-2-dataeval/logs",

    evaluation_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    metric_for_best_model = "loss",
    warmup_steps=15,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

In [31]:
from huggingface_hub import login

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")
#print(access_token_write)


login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [32]:
!pip install trl -q

In [33]:
!pip install rouge_score -q

  Preparing metadata (setup.py) ... done


In [34]:
import evaluate
# ----------------------------------------------------------------------------
# Evaluation Metrics
# ----------------------------------------------------------------------------

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[[l for l in label if l != -100]] for label in labels]
    true_predictions = [
        [[p for (p, l) in zip(pred, label) if l != -100]]
        for pred, label in zip(predictions, labels)
    ]
    bleu_result = bleu.compute(predictions=true_predictions, references=true_labels)
    rouge_result = rouge.compute(predictions=true_predictions, references=true_labels)

    print("Bleu:", bleu_result["bleu"])
    print("Rouge1:", rouge_result["rouge1"], "\n")
    print("Rouge2:", rouge_result["rouge2"], "\n")
    print("RougeL:", rouge_result["rougeL"], "\n")

    return {
        "bleu": bleu_result["bleu"],
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
    }


In [35]:
from transformers import EarlyStoppingCallback, DataCollatorForLanguageModeling

In [36]:
from bitsandbytes.nn import Linear8bitLt

In [37]:
# ----------------------------------------------------------------------------
# SFTTrainer Setup and Training (part 1)
# ----------------------------------------------------------------------------

# Create data collator to bucket the dataset
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [38]:
# ----------------------------------------------------------------------------
# SFTTrainer Setup and Training (part 2)
# ----------------------------------------------------------------------------


from trl import SFTTrainer
max_seq_length = 1024 # max sequence length for model and packing of the dataset
checkpointing_args = {"use_reentrant": False}

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    eval_dataset=dataset_test,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    #dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        #"dataset_text_field": "text",  # Specify the text field in the dataset
    },
    data_collator=data_collator,  # Use the data collator for dynamic padding
    #compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [39]:
# Enable gradient checkpointing on the model itself
model.gradient_checkpointing_enable()
model.gradient_checkpointing_kwargs = checkpointing_args

In [40]:
torch.cuda.empty_cache()

In [41]:
# ----------------------------------------------------------------------------
# SFTTrainer Setup and Training (Final)
# ----------------------------------------------------------------------------

from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))

# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

Test Model and run Inference

In [ ]:
torch.cuda.empty_cache()

In [ ]:
peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2-dataeval"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = '/content/gdrive/MyDrive/model/POC-Mistral-7B-text-to-sql-flash-attention-2-dataeval'
#peft_model_id = "frankmorales2020/POC-Mistral-7B-text-to-sql-flash-attention-2-dataeval"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
from tqdm import tqdm
from random import randint
from datasets import load_dataset

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

#rand_idx=6

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    if predicted_answer == sample["messages"][2]["content"]:
        return 1
    else:
        return 0

success_rate = []
number_of_eval_samples = 10
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)
#print()
#print(f"Accuracy: {accuracy*100:.2f}%")

In [ ]:
print()
#print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")